# [Module 4.2] getRec() 통한 추천 리스트 얻기

이 노트북은 Module2에서 생성한 솔류션을 바탕으로 아래와 같은 작업을 합니다.
* 캠페인 생성
* 캠페인을 통해 특정 유저에 대한 추천 영화 리스트 얻기


In [5]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [6]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [7]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

## 샘플 추천 결과 얻기

캠페인이 활성화되면 추천 결과를 받을 수 있습니다. 먼저 컬렉션에서 임의의 사용자를 선택해야 합니다. 그런 다음, ID 대신 추천을 위해 영화 정보를 표시하는 몇 가지 헬퍼 함수를 만듭니다.

In [8]:
items_all = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
items=items_all.copy()

items['to_keep'] = items['ITEM_ID'].apply(lambda x:x in unique_items)
items=items[items['to_keep']]
del items['to_keep']
items.tail()

#len(unique_items)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,ITEM_ID,TITLE,GENRE
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [9]:
def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=items[items['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())


#### HRNN GetRecommendations 호출

아래 코드 셀을 실행하면 특정 사용자에 대한 추천 사항이 표시되고 추천 영화 목록이 반환됩니다.

In [10]:
df=pd.read_csv(interaction_filename)

# Getting a random user:
user_id, item_id, _,_,_ = df.sample().values[0]
print("USER: {}".format(user_id))

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = hrnn_campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']
recommendation_title_list = []
recommendation_id_list=[]
for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_title_list.append(title)
    recommendation_id_list.append(item['itemId'])
recommendations_df = pd.DataFrame(recommendation_title_list, columns = ['OriginalRecs'])
recommendations_df

USER: 5047
Recommendations for user:  5047


,OriginalRecs
0,"Rock, The (1996)"
1,Screamers (1995)
2,Independence Day (ID4) (1996)
3,Stargate (1994)
4,Outbreak (1995)
5,Mimic (1997)
6,Face/Off (1997)
7,Scream 2 (1997)
8,Demolition Man (1993)
9,Star Wars: Episode I - The Phantom Menace (1999)


#### Sims GetRecommendations 호출
아래 코드 셀을 실행하면 특정 아이템과 유사한 추천 영화 목록이 반환됩니다.

In [11]:
# Getting a random user:
user_id, item_id, _,_,_ = df.sample().values[0]
print("ITEM ID: {}".format(item_id))


get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(item_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for item_id: ", item_id)

item_list = get_recommendations_response['itemList']
recommendation_title_list = []
recommendation_id_list=[]
for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_title_list.append(title)
    recommendation_id_list.append(item['itemId'])
recommendations_df = pd.DataFrame(recommendation_title_list, columns = ['OriginalRecs'])
recommendations_df

ITEM ID: 2160
Recommendations for item_id:  2160


,OriginalRecs
0,"Omen, The (1976)"
1,Psycho (1960)
2,"Exorcist, The (1973)"
3,Halloween (1978)
4,"Howling, The (1980)"
5,"Wolf Man, The (1941)"
6,Dracula (1931)
7,"American Werewolf in London, An (1981)"
8,"Haunting, The (1963)"
9,Frankenstein (1931)



## Personalized Ranking

Personalized Ranking의 핵심 사용 사례는 아이템 리스트를 가져 와서 사용자에게 우선 순위 또는 사용자 관심 순서로 표시하는 것입니다. 이 기능에 대해 알아보기 위해 이번 파트에서는 한명의 사용자와 랜덤하게 뽑은 25개의 아이템 항목을 가지고 테스트 해 봅니다. 

In [12]:
#Get the user list
df=pd.read_csv(interaction_filename)
df_users = df['USER_ID'].unique()
df_users=pd.DataFrame(df_users,columns=['USER_ID'])
df_items=df['ITEM_ID'].unique()
df_items=pd.DataFrame(df_items,columns=['ITEM_ID'])

rerank_user = df_users['USER_ID'].sample(1).tolist()[0]
rerank_items = df_items['ITEM_ID'].sample(25).tolist()

In [13]:
rerank_list = []
for item in rerank_items:
    title = get_movie_title(item)
    rerank_list.append(title)
rerank_df = pd.DataFrame(rerank_list, columns = [rerank_user])
rerank_df


,988
0,Larger Than Life (1996)
1,Meatballs Part II (1984)
2,Ninotchka (1939)
3,"Devil and Max Devlin, The (1981)"
4,Friday (1995)
5,Chungking Express (1994)
6,Raiders of the Lost Ark (1981)
7,Rhyme & Reason (1997)
8,"Battleship Potemkin, The (Bronenosets Potyomki..."
9,One Night Stand (1997)


In [14]:
# Convert user to string:
user_id = str(rerank_user)

rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = ranking_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

get_recommendations_response_rerank

{'ResponseMetadata': {'RequestId': 'b2288ed6-c87b-4f74-b66b-bdad655f2430',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Sun, 30 Aug 2020 12:51:13 GMT',
   'x-amzn-requestid': 'b2288ed6-c87b-4f74-b66b-bdad655f2430',
   'content-length': '1385',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'personalizedRanking': [{'itemId': '589', 'score': 0.9722281},
  {'itemId': '1198', 'score': 0.01182},
  {'itemId': '69', 'score': 0.0085482},
  {'itemId': '2011', 'score': 0.0065302},
  {'itemId': '936', 'score': 0.0002307},
  {'itemId': '165', 'score': 0.000125},
  {'itemId': '2844', 'score': 0.0001119},
  {'itemId': '3041', 'score': 0.0001039},
  {'itemId': '3235', 'score': 9.58e-05},
  {'itemId': '1668', 'score': 9.23e-05},
  {'itemId': '1652', 'score': 3.07e-05},
  {'itemId': '1470', 'score': 2.97e-05},
  {'itemId': '3119', 'score': 1.47e-05},
  {'itemId': '3490', 'score': 1.45e-05},
  {'itemId': '2044', 'score': 1.11e-05},
  {'itemId': '253

In [15]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    title = get_movie_title(item['itemId'])
    ranked_list.append(title)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,988,Re-Ranked
0,Larger Than Life (1996),Terminator 2: Judgment Day (1991)
1,Meatballs Part II (1984),Raiders of the Lost Ark (1981)
2,Ninotchka (1939),Friday (1995)
3,"Devil and Max Devlin, The (1981)",Back to the Future Part II (1989)
4,Friday (1995),Ninotchka (1939)
5,Chungking Express (1994),Die Hard: With a Vengeance (1995)
6,Raiders of the Lost Ark (1981),"Minus Man, The (1999)"
7,Rhyme & Reason (1997),Meatballs Part II (1984)
8,"Battleship Potemkin, The (Bronenosets Potyomki...",Where the Buffalo Roam (1980)
9,One Night Stand (1997),One Night Stand (1997)


## 리뷰

캠페인을 생성하고 실제적으로 특정 유저의 추천 영화 목록도 얻었습니다.
이제 다음 노트북으로 넘어갈 준비가 되었습니다. (`4.View_Campaign_And_Interactions.ipynb`)


## 다음 노트북에 대한 참고 사항

다음 실습에 필요한 몇 가지 값들이 있습니다. 아래 셀을 실행하여 저장한 후, 다음 주피터 노트북에서 그대로 사용할 수 있습니다.

In [16]:
%store recommendations_df
%store user_id

Stored 'recommendations_df' (DataFrame)
Stored 'user_id' (str)
